In [56]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/codebasics/deep-learning-keras-tf-tutorial/master/14_imbalanced/Handling%20Imbalanced%20Data%20In%20Customer%20Churn%20Using%20ANN/Churn_Modelling.csv')
df.head()

In [5]:
df.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

It's look there isn't any empty values. So lets see what data types available

In [6]:
df.dtypes

RowNumber            int64
CustomerId           int64
Surname             object
CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

Basic describe of numerical features

In [7]:
df.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


about categorical features

In [8]:
df.describe(include='O')

,Surname,Geography,Gender
count,10000,10000,10000
unique,2932,3,2
top,Smith,France,Male
freq,32,5014,5457


Lets see which categories are available in each column

In [9]:
for column in df:
    if df[column].dtype == 'object':
        print(f'{column}: {df[column].unique()}')

Surname: ['Hargrave' 'Hill' 'Onio' ... 'Kashiwagi' 'Aldridge' 'Burbidge']
Geography: ['France' 'Spain' 'Germany']
Gender: ['Female' 'Male']


There is lot more unique surname values. So lets see count

In [10]:
df['Surname'].value_counts()

Smith       32
Scott       29
Martin      29
Walker      28
Brown       26
            ..
Izmailov     1
Bold         1
Bonham       1
Poninski     1
Burbidge     1
Name: Surname, Length: 2932, dtype: int64

We want to group all the surname which has less than five count 

In [11]:
df1 = df.copy()

In [12]:
surname_stats = df1['Surname'].value_counts()
surname_stats_lessthan_5 = surname_stats[surname_stats < 5]
len(surname_stats_lessthan_5)

2311

In [13]:
df1['Surname'] = df1['Surname'].apply(lambda x: 'Rare' if x in surname_stats_lessthan_5 else x)

In [14]:
df1.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Rare,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


We want to convert categorical features to numeric. So lets do that

In [15]:
df1['Gender'].replace({'Male': 1, 'Female':0}, inplace=True)

Lets get dummies for Surname and Geography columns

In [16]:
df2 = df1.copy()

In [17]:
df2 = pd.get_dummies(data=df2, columns=['Geography', 'Surname']) #, 'Surname'

In [18]:
df2.head()

,RowNumber,CustomerId,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,...,Surname_Yin,Surname_Yobachi,Surname_Yobachukwu,Surname_Yobanna,Surname_Young,Surname_Yu,Surname_Yuan,Surname_Zetticci,Surname_Zikoranachidimma,Surname_Zito
0,1,15634602,619,0,42,2,0.00,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,15647311,608,0,41,1,83807.86,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,3,15619304,502,0,42,8,159660.80,3,1,0,...,0,0,0,0,0,0,0,0,0,0
3,4,15701354,699,0,39,1,0.00,2,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,15737888,850,0,43,2,125510.82,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [19]:
df2.shape

(10000, 637)

Lets drop unwanted columns

In [20]:
df3 = df2.drop(['RowNumber', 'CustomerId'], axis=1)

In [21]:
df3.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,...,Surname_Yin,Surname_Yobachi,Surname_Yobachukwu,Surname_Yobanna,Surname_Young,Surname_Yu,Surname_Yuan,Surname_Zetticci,Surname_Zikoranachidimma,Surname_Zito
0,619,0,42,2,0.00,1,1,1,101348.88,1,...,0,0,0,0,0,0,0,0,0,0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,...,0,0,0,0,0,0,0,0,0,0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,...,0,0,0,0,0,0,0,0,0,0
3,699,0,39,1,0.00,2,0,0,93826.63,0,...,0,0,0,0,0,0,0,0,0,0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,...,0,0,0,0,0,0,0,0,0,0


Now we want to scale the data

In [26]:
cols_to_scale = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
scaler = MinMaxScaler()

In [27]:
df3[cols_to_scale] = scaler.fit_transform(df3[cols_to_scale])

In [28]:
df3.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,...,Surname_Yin,Surname_Yobachi,Surname_Yobachukwu,Surname_Yobanna,Surname_Young,Surname_Yu,Surname_Yuan,Surname_Zetticci,Surname_Zikoranachidimma,Surname_Zito
0,0.538,0,0.324324,0.2,0.000000,0.000000,1,1,0.506735,1,...,0,0,0,0,0,0,0,0,0,0
1,0.516,0,0.310811,0.1,0.334031,0.000000,0,1,0.562709,0,...,0,0,0,0,0,0,0,0,0,0
2,0.304,0,0.324324,0.8,0.636357,0.666667,1,0,0.569654,1,...,0,0,0,0,0,0,0,0,0,0
3,0.698,0,0.283784,0.1,0.000000,0.333333,0,0,0.469120,0,...,0,0,0,0,0,0,0,0,0,0
4,1.000,0,0.337838,0.2,0.500246,0.000000,1,1,0.395400,0,...,0,0,0,0,0,0,0,0,0,0


Now we want to round float numbers to about 5 decimal points

In [29]:
for column in df3:
    df3[column] = round(df3[column], 5)

Split the data

In [30]:
X = df3.drop('Exited', axis='columns')
y = df3['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((8000, 634), (2000, 634), (8000,), (2000,))

Lets build the model

In [44]:
def ann(X_train, X_test, y_train, y_test):
    model = keras.Sequential([
        keras.layers.Dense(634, input_shape=(634,), activation='relu'),
        keras.layers.Dense(300, activation='relu'),
        keras.layers.Dense(50, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer = 'adam',
        loss= 'binary_crossentropy',
        metrics = ['accuracy']
    )

    model.fit(X_train, y_train, epochs=15)
    print('Accuracy :',model.evaluate(X_test, y_test))
    yp = model.predict(X_test)
    y_pred = []
    for element in yp:
        if element > 0.5:
            y_pred.append(1)
        else:
            y_pred.append(0)
    return y_pred

In [33]:
y_pred = ann(X_train, X_test, y_train, y_test)
y_pred

63/63 [==============================] - 1s 6ms/step


array([[2.4916690e-03],
       [2.0797856e-01],
       [8.3727700e-01],
       ...,
       [1.4375421e-01],
       [8.1462211e-05],
       [9.8429179e-01]], dtype=float32)

In [35]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.91      0.88      1600
           1       0.53      0.41      0.46       400

    accuracy                           0.81      2000
   macro avg       0.69      0.66      0.67      2000
weighted avg       0.79      0.81      0.80      2000



## __Imbalance data__

In [46]:
df3.Exited.value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

01. __Under sampling__

In [47]:
count_class_0, count_class_1 = df3.Exited.value_counts()

df_class_0 = df3[df3['Exited'] == 0]
df_class_1 = df3[df3['Exited'] == 1]

In [48]:
df_class_0_under = df_class_0.sample(count_class_1)
df_y_under = pd.concat([df_class_0_under, df_class_1], axis=0)
df_y_under.Exited.value_counts()

0    2037
1    2037
Name: Exited, dtype: int64

In [ ]:
X = df_y_under.drop('Exited', axis=1)
y = df_y_under['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

y_pred = ann(X_train, X_test, y_train, y_test)

In [50]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.67      0.75      0.71       408
           1       0.72      0.64      0.67       407

    accuracy                           0.69       815
   macro avg       0.69      0.69      0.69       815
weighted avg       0.69      0.69      0.69       815



02. __Over Sampling__

In [53]:
df_class_1_over = df_class_1.sample(count_class_0, replace=True)
df_over = pd.concat([df_class_1_over, df_class_0], axis=0)

df_over.Exited.value_counts()

1    7963
0    7963
Name: Exited, dtype: int64

In [ ]:
X = df_over.drop('Exited', axis=1)
y = df_over['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

y_pred = ann(X_train, X_test, y_train, y_test)

In [55]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.85      0.89      1593
           1       0.86      0.93      0.90      1593

    accuracy                           0.89      3186
   macro avg       0.89      0.89      0.89      3186
weighted avg       0.89      0.89      0.89      3186



3. __Oversample with SMOTE__

In [57]:
smote = SMOTE(sampling_strategy='minority')
x_sm, y_sm = smote.fit_resample(df3.drop('Exited', axis=1), df3['Exited'])

y_sm.value_counts()

1    7963
0    7963
Name: Exited, dtype: int64

In [58]:
X = x_sm
y = y_sm

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

y_pred = ann(X_train, X_test, y_train, y_test)

Epoch 1/15
399/399 [==============================] - 5s 11ms/step - loss: 0.5042 - accuracy: 0.7498
Epoch 2/15
399/399 [==============================] - 4s 11ms/step - loss: 0.3601 - accuracy: 0.8367
Epoch 3/15
399/399 [==============================] - 4s 11ms/step - loss: 0.2880 - accuracy: 0.8700
Epoch 4/15
399/399 [==============================] - 5s 11ms/step - loss: 0.2409 - accuracy: 0.8886
Epoch 5/15
399/399 [==============================] - 5s 12ms/step - loss: 0.2092 - accuracy: 0.9053
Epoch 6/15
399/399 [==============================] - 5s 11ms/step - loss: 0.1958 - accuracy: 0.9094
Epoch 7/15
399/399 [==============================] - 4s 11ms/step - loss: 0.1812 - accuracy: 0.9166
Epoch 8/15
399/399 [==============================] - 4s 11ms/step - loss: 0.1714 - accuracy: 0.9203
Epoch 9/15
399/399 [==============================] - 5s 12ms/step - loss: 0.1671 - accuracy: 0.9239
Epoch 10/15
399/399 [==============================] - 5s 12ms/step - loss: 0.1584 - accura

In [59]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.86      0.86      1593
           1       0.86      0.86      0.86      1593

    accuracy                           0.86      3186
   macro avg       0.86      0.86      0.86      3186
weighted avg       0.86      0.86      0.86      3186



4. __Ensemble method__

In [60]:
df3.Exited.value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [65]:
X = df3.drop(['Exited'], axis=1)
y = df3['Exited']

In [68]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

df_train = X_train.copy()
df_train['Exited'] = y_train

df_train_class_0 = df_train[df_train['Exited'] == 0]
df_train_class_1 = df_train[df_train['Exited'] == 1]

In [69]:
def get_train_batch(df_majority, df_minority, start, end):
    df_train = pd.concat([df_majority[start:end], df_minority], axis=0)

    X_train = df_train.drop('Exited', axis=1)
    y_train = df_train['Exited']

    return X_train, y_train

In [74]:
X_train, y_train = get_train_batch(df_train_class_0, df_train_class_1, 0, 2036)
y_pred1 = ann(X_train, X_test, y_train, y_test)

Epoch 1/15
115/115 [==============================] - 2s 11ms/step - loss: 0.6217 - accuracy: 0.6585
Epoch 2/15
115/115 [==============================] - 1s 10ms/step - loss: 0.5409 - accuracy: 0.7343
Epoch 3/15
115/115 [==============================] - 1s 12ms/step - loss: 0.4513 - accuracy: 0.7853
Epoch 4/15
115/115 [==============================] - 1s 11ms/step - loss: 0.3542 - accuracy: 0.8402
Epoch 5/15
115/115 [==============================] - 1s 10ms/step - loss: 0.2783 - accuracy: 0.8756
Epoch 6/15
115/115 [==============================] - 1s 12ms/step - loss: 0.2301 - accuracy: 0.8909
Epoch 7/15
115/115 [==============================] - 1s 10ms/step - loss: 0.2116 - accuracy: 0.8999
Epoch 8/15
115/115 [==============================] - 1s 11ms/step - loss: 0.1965 - accuracy: 0.9083
Epoch 9/15
115/115 [==============================] - 1s 11ms/step - loss: 0.1860 - accuracy: 0.9130
Epoch 10/15
115/115 [==============================] - 1s 11ms/step - loss: 0.1778 - accura

In [75]:
X_train, y_train = get_train_batch(df_train_class_0, df_train_class_1, 2036, 4072)
y_pred2 = ann(X_train, X_test, y_train, y_test)

Epoch 1/15
115/115 [==============================] - 2s 11ms/step - loss: 0.6285 - accuracy: 0.6536
Epoch 2/15
115/115 [==============================] - 1s 11ms/step - loss: 0.5419 - accuracy: 0.7275
Epoch 3/15
115/115 [==============================] - 1s 11ms/step - loss: 0.4512 - accuracy: 0.7921
Epoch 4/15
115/115 [==============================] - 1s 10ms/step - loss: 0.3589 - accuracy: 0.8393
Epoch 5/15
115/115 [==============================] - 1s 11ms/step - loss: 0.2848 - accuracy: 0.8759
Epoch 6/15
115/115 [==============================] - 1s 12ms/step - loss: 0.2317 - accuracy: 0.9015
Epoch 7/15
115/115 [==============================] - 1s 11ms/step - loss: 0.2063 - accuracy: 0.9100
Epoch 8/15
115/115 [==============================] - 1s 11ms/step - loss: 0.1901 - accuracy: 0.9149
Epoch 9/15
115/115 [==============================] - 1s 12ms/step - loss: 0.1835 - accuracy: 0.9146
Epoch 10/15
115/115 [==============================] - 1s 11ms/step - loss: 0.1721 - accura

In [76]:
X_train, y_train = get_train_batch(df_train_class_0, df_train_class_1, 4072, 6108)
y_pred3 = ann(X_train, X_test, y_train, y_test)

Epoch 1/15
115/115 [==============================] - 2s 10ms/step - loss: 0.6193 - accuracy: 0.6552
Epoch 2/15
115/115 [==============================] - 1s 10ms/step - loss: 0.5370 - accuracy: 0.7316
Epoch 3/15
115/115 [==============================] - 1s 11ms/step - loss: 0.4484 - accuracy: 0.7875
Epoch 4/15
115/115 [==============================] - 1s 11ms/step - loss: 0.3485 - accuracy: 0.8440
Epoch 5/15
115/115 [==============================] - 1s 10ms/step - loss: 0.2819 - accuracy: 0.8775
Epoch 6/15
115/115 [==============================] - 1s 10ms/step - loss: 0.2293 - accuracy: 0.8985
Epoch 7/15
115/115 [==============================] - 1s 10ms/step - loss: 0.2055 - accuracy: 0.9108
Epoch 8/15
115/115 [==============================] - 1s 10ms/step - loss: 0.1888 - accuracy: 0.9152
Epoch 9/15
115/115 [==============================] - 1s 10ms/step - loss: 0.1809 - accuracy: 0.9203
Epoch 10/15
115/115 [==============================] - 1s 12ms/step - loss: 0.1735 - accura

In [78]:
X_train, y_train = get_train_batch(df_train_class_0, df_train_class_1, 6108, 7963)
y_pred4 = ann(X_train, X_test, y_train, y_test)

Epoch 1/15
60/60 [==============================] - 1s 11ms/step - loss: 0.3998 - accuracy: 0.8610
Epoch 2/15
60/60 [==============================] - 1s 12ms/step - loss: 0.3330 - accuracy: 0.8615
Epoch 3/15
60/60 [==============================] - 1s 11ms/step - loss: 0.2766 - accuracy: 0.8864
Epoch 4/15
60/60 [==============================] - 1s 11ms/step - loss: 0.2189 - accuracy: 0.9160
Epoch 5/15
60/60 [==============================] - 1s 13ms/step - loss: 0.1786 - accuracy: 0.9276
Epoch 6/15
60/60 [==============================] - 1s 9ms/step - loss: 0.1524 - accuracy: 0.9466
Epoch 7/15
60/60 [==============================] - 1s 11ms/step - loss: 0.1345 - accuracy: 0.9466
Epoch 8/15
60/60 [==============================] - 1s 13ms/step - loss: 0.1222 - accuracy: 0.9508
Epoch 9/15
60/60 [==============================] - 1s 10ms/step - loss: 0.1191 - accuracy: 0.9519
Epoch 10/15
60/60 [==============================] - 1s 10ms/step - loss: 0.1079 - accuracy: 0.9582
Epoch 11/1

In [79]:
y_pred_final = y_pred1.copy()
for i in range(len(y_pred1)):
    n_ones = y_pred1[i] + y_pred2[i] + y_pred3[i]+ y_pred4[i]
    if n_ones > 1:
        y_pred_final[i] = 1
    else:
        y_pred_final[i] = 0

In [80]:
print(classification_report(y_test, y_pred_final))

              precision    recall  f1-score   support

           0       0.92      0.62      0.74      1593
           1       0.34      0.77      0.48       407

    accuracy                           0.65      2000
   macro avg       0.63      0.70      0.61      2000
weighted avg       0.80      0.65      0.69      2000

